### Coursework 2

In this coursework you will be aiming to complete two classification tasks. 
Both the classification tasks relate to text classification tasks. 

One task is to be solved using Support Vector Machines. The other has to be solved using Boosting.

The specific tasks and the marking for the various tasks are provided in the notebook. Each task is expected to be accompanied by a lab-report. Each task can have a concise lab report that is maximum of one page in an A4 size. You will be expected to submit your Jupyter Notebook and all lab reports as a single zip file. You could have additional functions implemented that you require for carrying out each task.


#### Task 1

In this task, you need to obtain sentiment analysis for the provided dataset. The dataset consists of movie reviews with the sentiments being provided. The sentiments are either positive or negative. You need to train an SVM based classifier to obtain train and check on the sample test dataset provided. The method will be evaluated also against an external test set. Please do not hardcode any dimensions or number of samples while writing the code. It should be possible to automate the testing and hardcoding values does not allow for automated testing. 

You are allowed to use scikit-learn to implement the SVM. However, you are expected to write your own kernels.

You are allowed to use the existing library functions such as scikit-learn or numpy for obtaining the SVM. The main idea is to analyse the dataset using different kind of kernels. You are also supposed to write your own custom text kernels. Refer to the documentation provided [here](https://scikit-learn.org/stable/modules/svm.html) at 1.4.6.2 and an example [here](https://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html) for writing your own kernels.

Details regarding the marking have been provided in the coursework specification file. Ensure that the code can be run with different test files. 

#### Process the text and obtain a bag of words-based features 

In [99]:
nltk.download('stopwords')
nltk.download('words')
nltk.download('brown')
    
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import jaccard_distance
from bs4 import BeautifulSoup
from nltk.util import ngrams
from nltk.corpus import brown
from nltk import FreqDist
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

'''
correct_words = set(w.lower() for w in brown.words())
freq_list = FreqDist(w for w in correct_words)

def spell_correct(word): # not complete
    
    if word not in correct_words:
        potential_list = [(jaccard_distance(set(ngrams(word, 2)), set(ngrams(w, 2))),w) for w in correct_words if w[0]==word[0]]
        print(word)
        print(np.array(potential_list)[:,1])
        
        word = potential_list[0][1]
    
    return word
'''

def pre_processing(dataset):
    
    for i in range(len(dataset)):
        #print(i)
        # remove html tags
        dataset[i] = BeautifulSoup(dataset[i]).get_text()
        
        # convert to lower case
        dataset[i] = dataset[i].lower()
        
        # tokenize
        dataset[i] = word_tokenize(dataset[i])
                
        # remove punctuation
        dataset[i] = [word for word in dataset[i] if word.isalpha()]
        
        # remove lengthened words e.g. finallllly
        pattern = re.compile(r"(.)\1{2,}")
        dataset[i] = [pattern.sub(r"\1\1", word) for word in dataset[i]]
        
        # spell correct - implement if performance is not good enough
        #dataset[i] = [spell_correct(word) for word in dataset[i]]
        
        # remove stop words
        for word in dataset[i]:
            if word in stopwords.words('english'):
                dataset[i].remove(word)
        
        # stemming
        stemmer = PorterStemmer()
        dataset[i] = [stemmer.stem(word) for word in dataset[i]]
        
        # join words with space
        dataset[i] = ' '.join(dataset[i])
             
    return dataset

def extract_bag_of_words_train_test(train_file, test_file):
    
    # Read the CSV files for training and test sets
    train = pd.read_csv(train_file)
    test = pd.read_csv(test_file)
    
    X_train = np.array(train.review)
    X_test = np.array(test.review)
    y_train = np.array(train.sentiment)
    y_test = np.array(test.sentiment)
    
    # Extract bag of words features
    
    X_train = pre_processing(X_train) 
    print('Train set: done processing data')
    X_test = pre_processing(X_test)
    print('Test set: done processing data')
    
    # get feature vectors
    tf_idf = TfidfVectorizer()
    X_train = tf_idf.fit_transform(X_train)
    print('Train set: done getting feature vector')
    X_test = tf_idf.transform(X_test)
    print('Test set: done getting feature vector')
    
    return (X_train,y_train,X_test,y_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Sayuri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /Users/Sayuri/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to /Users/Sayuri/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [95]:
from sklearn import svm

class SVMClassifier:
    def __init__(self):
        
        #implement initialisation
        self.kernel = 'linear'
        
        # for cross-validation 
        ## to be implemented 
        self.train_val_split = 0.8
        self.num_fold = 5 
        
        # for linear
        
    # define your own kernel here
    # Refer to the documentation here: https://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html
    def linear_kernel(X, Y):
        return np.dot(X, Y.T)
    
    def RBF_kernel(): # to be implemented
        return
        
    def polynomial_kernel(): # to be implemented
        return
    
    def sigmoid_kernel(): # to be implemented
        return
    
    def fit(self, X,y):
        # training of the SVM
        # Ensure you call your own defined kernel here
        if self.kernel == 'linear':
            self.clf = svm.SVC(self.linear_kernel)
            
        else if self.kernel == 'rbf':
            self.clf = svm.SVC(self.RBF_kernel)
            
        else if self.kernel == 'polynomial':
            self.clf = svm.SVC(self.polynomial_kernel)
        
        else if self.kernel == 'sigmoid':
            self.clf = svm.SVC(self.sigmoid_kernel)
            
        return self.clf.fit(X,y)
    
    def predict(self, X):
        # prediction routine for the SVM
        return self.clf.fit(X,y)

In [ ]:
# self testing code - remove before submission
(X_train, Y_train, X_test, Y_test) = extract_bag_of_words_train_test("movie_review_train.csv", "movie_review_test.csv")

In [ ]:
# self testing code - remove before submission
from sklearn.metrics import accuracy_score  
sc = SVMClassifier()
sc.fit(X_train, Y_train)
Y_Pred = sc.predict(X_test)
acc = accuracy_score(Y_test, Y_Pred)
print("Accuracy:",acc)

### Test function that will be called to evaluate your code. Separate test dataset will be provided

Do not modify the code below. Please write your code above such that it can be evaluated by the function below. You can modify your code above such that you obtain the best performance through this function. 

In [98]:
def test_func_svm(dataset_train, dataset_test):
    from sklearn.metrics import accuracy_score  
    (X_train, Y_train, X_test, Y_test) = extract_bag_of_words_train_test(dataset_train, dataset_test)
    sc = SVMClassifier()
    sc.fit(X_train, Y_train)
    Y_Pred = sc.predict(X_test)
    acc = accuracy_score(Y_test, Y_Pred)
    print("Accuracy:",acc)
    return acc

In [100]:
acc = test_func_svm("movie_review_train.csv", "movie_review_test.csv")

NameError: name 'linear_kernel' is not defined

### Task 2

In this task you need to implement a boosting based classifier that can be used to classify the images. 

Details regarding the marking for the coursework are provided in the coursework specification file. Please ensure that your code will work with a different test file than the one provided with the coursework.

Note that the boosting classifier you implement can include decision trees from scikit-learn or your own decision trees. Use the same sentiment analysis dataset for evaluation.

In [4]:
class BoostingClassifier:
    # You need to implement this classifier. 
    def __init__(self):
        import numpy as np
        #implement initialisation
        self.some_paramter=1
    def fit(self, X,y):
        from sklearn.tree import DecisionTreeClassifier
        import numpy as np
        #implement training of the boosting classifier
        return 
    def predict(self, X):
        # implement prediction of the boosting classifier
        return

### Test function that will be called to evaluate your code. Separate test dataset will be provided

Do not modify the code below. Please write your code above such that it can be evaluated by the function below. You can modify your code above such that you obtain the best performance through this function. 

In [5]:
def test_func_boosting(dataset_train, dataset_test):
    from sklearn.metrics import accuracy_score    
    (X_train, Y_train, X_test, Y_test) = extract_bag_of_words_train_test(dataset_train, dataset_test)
    bc = BoostingClassifier()
    bc.fit(X_train, Y_train)
    Y_Pred = bc.predict(X_test)    
    acc = accuracy_score(Y_test, Y_Pred)
    return acc

In [ ]:
acc = test_func_boosting("movie_review_train.csv", "movie_review_test.csv")